In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

# 1. 모델과 토크나이저를 4비트(4-bit)로 로드합니다.
#    최대 시퀀스 길이를 1024로 설정하여 VRAM 사용량을 줄입니다.
max_seq_length = 1024
model_name = "unsloth/gpt-oss-7b" # 더 작은 모델로 빠른 테스트가 가능합니다.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True, # 4비트 양자화
    _initialize_weights = False, # 모델 가중치 초기화 오류 방지
)

# 2. 모델을 QLoRA에 맞게 설정합니다.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA 랭크
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # 메모리 절약
    random_state = 3407,
)

# 3. 데이터셋을 준비합니다. (alpaca 예시)
dataset = load_dataset("yahma/alpaca-cleaned", split="train")

def formatting_prompts_func(examples):
    instruction = "주어진 텍스트를 긍정, 부정으로 분류하세요." # 분류 작업에 대한 지시문
    texts = []
    for text, label in zip(examples["text"], examples["label"]):
        prompt = f"### Instruction:\n{instruction}\n\n### Text:\n{text}\n\n### Classification:\n{label}"
        texts.append(prompt)
    return {"text": texts,}

# 기존 노트북 코드의 이 부분만 수정하면 됩니다.
dataset = load_dataset("your_text_classification_dataset.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# 4. 학습 매개변수를 설정하고 학습을 시작합니다.
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 1, # VRAM 최적화를 위한 배치 사이즈
        gradient_accumulation_steps = 4, # 효과적인 배치 사이즈 4
        max_steps = 100, # 예시 학습 스텝 수
        learning_rate = 2e-4,
        bf16 = True,
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

# 5. 파인튜닝된 모델을 저장합니다.
model.save_pretrained("lora_model", push_to_hub=False)